**Задание**: составить список всех n-грамм токенов из текста, длиной не больше заданного пользователем значения (то есть, всех последовательностей из 1, 2, 3, …, k слов текста). Каждая n-грамма имеет собственный уникальный номер. Организовать хранение структуры в виде двунаправленного префиксного дерева. По номеру фразы выдавать ее ?номер?.

In [364]:
import re

#### Задаём структуру: класс Tree() и класс Node()
Пусть каждая вершина дерева будет являться объектом класса Node() с атрибутами index, token, parent и children, где index - индекс n-граммы, заканчивающейся на этом слове, token - токен в этой вершине, parent - index родительской вершины, а children - словарь объектов класса Node-детей этой вершины.

In [375]:
class Node:
    def __init__(self):
        self.index = None
        self.token = None
        self.parent = None
        self.children = {}

In [482]:
class Tree:
    def __init__(self):  # создаём объект класса
        self.root = Node()  # задаваемый корневой вершиной, у которой все поля пока что пустые
        self.num_of_ngrams = 0  # для удобства после генерации дерева будем хранить число n-грамм в нём
        self.leaves = []

    
    def ngrams_numbered(self, text: str, n: int):  # метод генерации n-грамм текста
        ngrams = []
        numbered = []
        words = []
        newtext = re.sub(r'[^\w\s]', '', text)  # чистим текст от пунктуации
        for x in newtext.split():
            words.append(x.lower())  # список слов текста
        c = 1  # счётчик для индексов n-грамм
        amount = min(n, len(words))  # следим, чтобы длина требуемых n-грамм не вылезала за длину текста
        for x in range(1, amount + 1):
            for i in range(len(words) + 1 - x):
                adding = tuple(words[i:i + x])  # n-граммы - срезы списка слов текста
                if adding not in ngrams:  # следим за уникальностью n-грамм
                    ngrams.append(adding)
                    numbered.append([c, adding])  # храним n-грамму как список из номера и кортежа слов n-граммы
                    c += 1
        return numbered
        

    def add(self, ngram: list, node: dict):  # метод добавления n-грамм в дерево из результатат ngrams_numbered. Рекурсивно обходим дерево от корня
        if ngram[1][0] not in node.children.keys(): # если на каком-то узле мы обнаруживаем, что следующего слова n-граммы нет среди детей-вершин
            word = ngram[1][0]
            node.children[word] = Node()
            node.children[word].index = ngram[0]
            node.children[word].token = word
            node.children[word].parent = node.index
            node.children[word].children = {}
    # так как ngrams_numbered хранит n-граммы упорядоченно от меньших к большим, за раз функция add не добавляет больше одной вершины - завершаем
            self.num_of_ngrams += 1
            self.leaves.append(node.children[word])
            return node.children[word]
        else:    
            return self.add([ngram[0], ngram[1][1:]], node.children[ngram[1][0]])  # перемещаемся вниз, если следующее слово есть среди детей


    def generate_tree(self, text: str, n: int):  # объединяющий два предыдущих метод: генерация дерева из текста
        numbered = self.ngrams_numbered(text, n)
        for x in numbered:
            self.add(x, self.root)
        print(f'The tree is ready! Number of ngrams: {self.num_of_ngrams}')

    
    def find_with_parent(self, index: int, phrase=[]):  # метод выдачи n-граммы по индексу путём обнаружения последнего слова и дороги до родителей
        for x in self.leaves:  # проходимся по ссылкам в списке-атрибуте дерева
            if x.index == index:  
                newphrase = [x.token] + phrase  # набираем список-n-грамму
                if x.parent is not None:
                    result = self.find_with_parent(x.parent, phrase=newphrase)  # ранние слова n-граммы - рекурсивно относительно родителя последующего
                    return result
                return [x.token] + phrase               
                    
        
    def find_by_index(self, index, node=None, phrase=None):  # метод выдачи n-граммы по индексу перебором всего дерева
        
        if node is None:  # начальная точка прохода рекурсией
            node = self.root
            phrase = []  # а сюда набираем n-грамму
        
        if node.index == index:  # если дошли до нужной точки, то возвращаем набранную n-грамму
            return phrase
        
        for word, child in node.children.items():
            result = self.find_by_index(index, child, phrase + [word])  # перебираем все веточки дерева, добавляя слова-названия узлов в n-грамму  
            if result is not None:  # в большинстве выдача будет None, как командуем ниже; но в итоге найдётся нужный индекс
                return result
        
        return None

#### Пример работы

In [495]:
new = Tree()
test = "Все, кто помнит этот ветер, закатайте рукава (и закатайте штанины)"

In [496]:
testlist = new.ngrams_numbered(test, 10)

In [497]:
testlist

[[1, ('все',)],
 [2, ('кто',)],
 [3, ('помнит',)],
 [4, ('этот',)],
 [5, ('ветер',)],
 [6, ('закатайте',)],
 [7, ('рукава',)],
 [8, ('и',)],
 [9, ('штанины',)],
 [10, ('все', 'кто')],
 [11, ('кто', 'помнит')],
 [12, ('помнит', 'этот')],
 [13, ('этот', 'ветер')],
 [14, ('ветер', 'закатайте')],
 [15, ('закатайте', 'рукава')],
 [16, ('рукава', 'и')],
 [17, ('и', 'закатайте')],
 [18, ('закатайте', 'штанины')],
 [19, ('все', 'кто', 'помнит')],
 [20, ('кто', 'помнит', 'этот')],
 [21, ('помнит', 'этот', 'ветер')],
 [22, ('этот', 'ветер', 'закатайте')],
 [23, ('ветер', 'закатайте', 'рукава')],
 [24, ('закатайте', 'рукава', 'и')],
 [25, ('рукава', 'и', 'закатайте')],
 [26, ('и', 'закатайте', 'штанины')],
 [27, ('все', 'кто', 'помнит', 'этот')],
 [28, ('кто', 'помнит', 'этот', 'ветер')],
 [29, ('помнит', 'этот', 'ветер', 'закатайте')],
 [30, ('этот', 'ветер', 'закатайте', 'рукава')],
 [31, ('ветер', 'закатайте', 'рукава', 'и')],
 [32, ('закатайте', 'рукава', 'и', 'закатайте')],
 [33, ('рукава', 

In [498]:
new.generate_tree(test, 10)

The tree is ready! Number of ngrams: 54


In [499]:
new.leaves

In [505]:
new.find_by_index(34)

['все', 'кто', 'помнит', 'этот', 'ветер']

In [506]:
who = new.find_with_parent(34)
print(who)

['все', 'кто', 'помнит', 'этот', 'ветер']
